In [1]:
import pandas as pd
import numpy as np

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/home/xiasilo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../TestB/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../TestB/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../TestB/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('../data/uid_test_b.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

##  baseline 

### 通话记录

#### 通话时长

In [6]:
# return the talk time of the call(seconds)
def calTime(start, end):
    result = []
    for i in range(len(start)):
        day_s = int(start[i][0:2])
        hour_s = int(start[i][2:4])
        min_s = int(start[i][4:6])
        sec_s = int(start[i][6:8])
        day_e = int(end[i][0:2])
        hour_e = int(end[i][2:4])
        min_e = int(end[i][4:6])
        sec_e = int(end[i][6:8])
        talkTime = 0
        # seconds
        if sec_e < sec_s:
            talkTime += sec_e + 60 - sec_s
            min_e = min_e - 1
        else:
            talkTime += sec_e - sec_s
        
        # minute
        if min_e < min_s:
            talkTime += (min_e + 60 - min_s) * 60
            hour_e = hour_e - 1
        else:
            talkTime += (min_e - min_s) * 60
        
        # hour
        if hour_e < hour_s:
            talkTime += (hour_e + 24 - hour_s) * 60 * 60
            day_e = day_e - 1
        else:
            talkTime += (hour_e - hour_s) * 60 * 60
        
        # day
        if hour_e < hour_s:
            result.append(-1) 
        else:
            talkTime += (hour_e - hour_s) * 24 * 60 * 60
            result.append(talkTime)
    return result

In [7]:
voice['talk_time'] = calTime(voice['start_time'].values, voice['end_time'].values)
voice_talk_time = voice.groupby(['uid'])['talk_time'].agg(['count','std','max','min','median','mean','sum']).add_prefix('voice_talk_time_').reset_index()

In [8]:
def calcount(time, long):
    count = 0
    if time > long:
        count += 1
    return count

In [18]:
voice['less_than_5'] = voice['talk_time'] <= 5
voice['less_than_5'] = voice['less_than_5'].map({True: 1, False: 0})
voice['less_than_7200'] = voice['talk_time'] > 5 and voice['talk_time'] <= 7200
voice['less_than_7200'] = voice['less_than_7200'].map({True: 1, False: 0})
voice['less_than_18000'] = voice['talk_time'] > 7200 and voice['talk_time'] <= 18000
voice['less_than_18000'] = voice['less_than_18000'].map({True: 1, False: 0})
voice['less_than_36000'] = voice['talk_time'] > 18000 and voice['talk_time'] <= 36000
voice['less_than_36000'] = voice['less_than_36000'].map({True: 1, False: 0})
voice['more_than_36000'] = voice['talk_time'] > 36000
voice['more_than_36000'] = voice['less_than_36000'].map({True: 1, False: 0})

In [21]:
islessthan5 = voice.groupby(['uid', 'less_than_5'])['uid'].count().unstack().reset_index().fillna(0)
islessthan7200 = voice.groupby(['uid', 'less_than_7200'])['uid'].count().unstack().reset_index().fillna(0)
islessthan18000 = voice.groupby(['uid', 'less_than_18000'])['uid'].count().unstack().reset_index().fillna(0)
islessthan36000 = voice.groupby(['uid', 'less_than_36000'])['uid'].count().unstack().reset_index().fillna(0)
ismorethan36000 = voice.groupby(['uid', 'more_than_36000'])['uid'].count().unstack().reset_index().fillna(0)

voice_talk_time['voice_talk_time_less_than_5'] = islessthan5[1]
voice_talk_time['voice_talk_time_less_than_7200'] = islessthan7200[1]
voice_talk_time['voice_talk_time_less_than_18000'] = islessthan18000[1]
voice_talk_time['voice_talk_time_less_than_36000'] = islessthan36000[1]
voice_talk_time['voice_talk_time_more_than_36000'] = ismorethan36000[1]
voice_talk_time.head()

,uid,voice_talk_time_count,voice_talk_time_std,voice_talk_time_max,voice_talk_time_min,voice_talk_time_median,voice_talk_time_mean,voice_talk_time_sum,voice_talk_time_more_than_60,voice_talk_time_less_than_5
0,u0001,79,267.850118,1800,5,36.0,118.556962,9366,28.0,0.0
1,u0002,2,13.435029,39,20,29.5,29.500000,59,0.0,0.0
2,u0003,21,98.055912,336,6,33.0,83.190476,1747,8.0,0.0
3,u0004,254,398.162097,1800,1,43.0,202.003937,51309,101.0,4.0
4,u0005,401,66.933882,1238,1,27.0,38.832918,15572,55.0,1.0


In [25]:
voice.head()

,uid,opp_num,opp_head,opp_len,start_time,end_time,call_type,in_out,talk_time,less_than_5,more_than_60,start_day
0,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115956,26120033,1,1,37,0,0,26
1,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115623,26115707,1,1,44,0,0,26
2,u0113,38D54642A237A11BB18455FC1E505292,132,11,26174233,26174321,1,1,48,0,0,26
3,u0113,38D54642A237A11BB18455FC1E505292,132,11,26070423,26070512,1,0,49,0,0,26
4,u3340,010A66F2AD42C48C44897A3DEC96A2A1,139,11,26201745,26201825,1,1,40,0,0,26


#### 日通话次数

In [24]:
voice['start_day'] = voice['start_time'].str[0:2]

In [29]:
voice_day_talk = voice.groupby(['uid', 'start_day'])['uid'].count().unstack().add_prefix('voice_day_talk_').reset_index().fillna(0)

In [32]:
voice_day_talk.head()

start_day,uid,voice_day_talk_00,voice_day_talk_01,voice_day_talk_02,voice_day_talk_03,voice_day_talk_04,voice_day_talk_05,voice_day_talk_06,voice_day_talk_07,voice_day_talk_08,...,voice_day_talk_36,voice_day_talk_37,voice_day_talk_38,voice_day_talk_39,voice_day_talk_40,voice_day_talk_41,voice_day_talk_42,voice_day_talk_43,voice_day_talk_44,voice_day_talk_45
0,u0001,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,...,3.0,1.0,0.0,4.0,3.0,2.0,2.0,1.0,0.0,0.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,u0003,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,u0004,0.0,5.0,1.0,9.0,6.0,6.0,2.0,2.0,5.0,...,3.0,23.0,8.0,10.0,8.0,14.0,17.0,11.0,5.0,6.0
4,u0005,0.0,8.0,3.0,7.0,3.0,9.0,6.0,10.0,12.0,...,0.0,6.0,5.0,13.0,18.0,7.0,8.0,14.0,4.0,11.0


In [69]:
voice_day_talk_values = voice_day_talk.drop('uid', axis=1).values

In [70]:
max_apps = []
min_apps = []
mean_apps = []
median_apps = []
std_apps = []
sum_apps = []
for i in voice_day_talk_values:
    max_apps.append(max(i))
    min_apps.append(min(i))
    mean_apps.append(np.mean(i))
    median_apps.append(np.median(i))
    std_apps.append(np.std(i))
    sum_apps.append(sum(i))

In [71]:
voice_day_talk['voice_day_talk_max'] = max_apps
voice_day_talk['voice_day_talk_min'] = min_apps
voice_day_talk['voice_day_talk_mean'] = mean_apps
voice_day_talk['voice_day_talk_median'] = median_apps
voice_day_talk['voice_day_talk_std'] = std_apps
voice_day_talk['voice_day_talk_std'].fillna(0)
voice_day_talk['voice_day_talk_sum'] = sum_apps
voice_day_talk.head()

start_day,uid,voice_day_talk_00,voice_day_talk_01,voice_day_talk_02,voice_day_talk_03,voice_day_talk_04,voice_day_talk_05,voice_day_talk_06,voice_day_talk_07,voice_day_talk_08,...,voice_day_talk_42,voice_day_talk_43,voice_day_talk_44,voice_day_talk_45,voice_day_talk_max,voice_day_talk_min,voice_day_talk_mean,voice_day_talk_median,voice_day_talk_std,voice_day_talk_sum
0,u0001,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,7.0,...,2.0,1.0,0.0,0.0,8.0,0.0,1.717391,1.0,1.907229,79.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.043478,0.0,0.203931,2.0
2,u0003,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,4.0,0.0,0.456522,0.0,0.771969,21.0
3,u0004,0.0,5.0,1.0,9.0,6.0,6.0,2.0,2.0,5.0,...,17.0,11.0,5.0,6.0,23.0,0.0,5.521739,5.0,4.431798,254.0
4,u0005,0.0,8.0,3.0,7.0,3.0,9.0,6.0,10.0,12.0,...,8.0,14.0,4.0,11.0,23.0,0.0,8.717391,8.0,4.816922,401.0


#### 其他

In [33]:
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)


/home/xiasilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/xiasilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
voice_call_type.head()

call_type,uid,voice_call_type_1,voice_call_type_2,voice_call_type_3,voice_call_type_4,voice_call_type_5
0,u0001,79.0,0.0,0.0,0.0,0.0
1,u0002,2.0,0.0,0.0,0.0,0.0
2,u0003,21.0,0.0,0.0,0.0,0.0
3,u0004,218.0,6.0,30.0,0.0,0.0
4,u0005,398.0,0.0,3.0,0.0,0.0


## 短信记录

In [36]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)


sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


/home/xiasilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/xiasilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
sms['start_day'] = sms['start_time'].str[0:2]

In [40]:
sms_day_send = sms.groupby(['uid','start_day'])['uid'].count().unstack().add_prefix('sms_day_send_').reset_index().fillna(0)

In [41]:
sms_day_send.head()

start_day,uid,sms_day_send_01,sms_day_send_02,sms_day_send_03,sms_day_send_04,sms_day_send_05,sms_day_send_06,sms_day_send_07,sms_day_send_08,sms_day_send_09,...,sms_day_send_36,sms_day_send_37,sms_day_send_38,sms_day_send_39,sms_day_send_40,sms_day_send_41,sms_day_send_42,sms_day_send_43,sms_day_send_44,sms_day_send_45
0,u0001,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,5.0,...,7.0,7.0,6.0,2.0,0.0,3.0,10.0,2.0,14.0,1.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,u0003,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,2.0,2.0
3,u0004,4.0,6.0,1.0,2.0,0.0,0.0,4.0,3.0,4.0,...,3.0,0.0,8.0,3.0,5.0,0.0,1.0,11.0,0.0,1.0
4,u0005,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,1.0,0.0,1.0,4.0,2.0,1.0


In [72]:
sms_day_send_values = sms_day_send.drop('uid', axis=1).values

In [73]:
max_apps = []
min_apps = []
mean_apps = []
median_apps = []
std_apps = []
sum_apps = []
for i in sms_day_send_values:
    max_apps.append(max(i))
    min_apps.append(min(i))
    mean_apps.append(np.mean(i))
    median_apps.append(np.median(i))
    std_apps.append(np.std(i))
    sum_apps.append(sum(i))

In [74]:
sms_day_send['sms_day_send_max'] = max_apps
sms_day_send['sms_day_send_min'] = min_apps
sms_day_send['sms_day_send_mean'] = mean_apps
sms_day_send['sms_day_send_median'] = median_apps
sms_day_send['sms_day_send_std'] = std_apps
sms_day_send['sms_day_send_std'].fillna(0)
sms_day_send['sms_day_send_sum'] = sum_apps
sms_day_send.head()

start_day,uid,sms_day_send_01,sms_day_send_02,sms_day_send_03,sms_day_send_04,sms_day_send_05,sms_day_send_06,sms_day_send_07,sms_day_send_08,sms_day_send_09,...,sms_day_send_42,sms_day_send_43,sms_day_send_44,sms_day_send_45,sms_day_send_max,sms_day_send_min,sms_day_send_mean,sms_day_send_median,sms_day_send_std,sms_day_send_sum
0,u0001,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,5.0,...,10.0,2.0,14.0,1.0,14.0,0.0,2.111111,1.0,3.092803,95.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.022222,0.0,0.147406,1.0
2,u0003,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,...,0.0,3.0,2.0,2.0,4.0,0.0,0.622222,0.0,0.995298,28.0
3,u0004,4.0,6.0,1.0,2.0,0.0,0.0,4.0,3.0,4.0,...,1.0,11.0,0.0,1.0,12.0,0.0,2.755556,2.0,2.937539,124.0
4,u0005,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,4.0,2.0,1.0,6.0,0.0,1.066667,1.0,1.420485,48.0


### 网站/APP记录

In [42]:
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()


up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

/home/xiasilo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [ ]:
up_flow_mean = np.mean(wa.up_flow)
up_flow_std = np.std(wa.up_flow).fillna(0)
wa['less_than_mean_minus_std'] = wa['up_flow'] <= max(mean-std, 0)
wa['less_than_mean_minus_std'] = wa['less_than_mean_minus_std'].map({True: 1, False: 0})
wa['less_than_mean_add_std'] = wa['up_flow'] > max(mean-std, 0) and wa['up_flow'] <= mean+std
wa['less_than_mean_add_std'] = wa['less_than_mean_add_std'].map({True: 1, False: 0})
wa['more_than_mean_add_std'] = wa['up_flow'] > mean+std
wa['more_than_mean_add_std'] = wa['more_than_mean_add_std'].map({True: 1, False: 0})

less_than_mean_minus_std = voice.groupby(['uid', 'less_than_mean_minus_std'])['uid'].count().unstack().reset_index().fillna(0)
less_than_mean_add_std = voice.groupby(['uid', 'less_than_mean_add_std'])['uid'].count().unstack().reset_index().fillna(0)
more_than_mean_add_std = voice.groupby(['uid', 'lmore_than_mean_add_std'])['uid'].count().unstack().reset_index().fillna(0)

wa_up_flow_B = pd.DataFrame()
wa_up_flow_B['uid'] = wa.uid
wa_up_flow_B['wa_up_flow_less_than_mean_minus_std'] = less_than_mean_minus_std[1]
wa_up_flow_B['wa_up_flow_less_than_mean_add_std'] = less_than_mean_add_std[1]
wa_up_flow_B['wa_up_flow_more_than_mean_add_std'] = more_than_mean_add_std[1]

In [ ]:
down_flow_mean = np.mean(wa.down_flow)
down_flow_std = np.std(wa.down_flow).fillna(0)
wa['less_than_mean_minus_std'] = wa['down_flow'] <= max(mean-std, 0)
wa['less_than_mean_minus_std'] = wa['less_than_mean_minus_std'].map({True: 1, False: 0})
wa['less_than_mean_add_std'] = wa['down_flow'] > max(mean-std, 0) and wa['down_flow'] <= mean+std
wa['less_than_mean_add_std'] = wa['less_than_mean_add_std'].map({True: 1, False: 0})
wa['more_than_mean_add_std'] = wa['down_flow'] > mean+std
wa['more_than_mean_add_std'] = wa['more_than_mean_add_std'].map({True: 1, False: 0})

less_than_mean_minus_std = wa.groupby(['uid', 'less_than_mean_minus_std'])['uid'].count().unstack().reset_index().fillna(0)
less_than_mean_add_std = wa.groupby(['uid', 'less_than_mean_add_std'])['uid'].count().unstack().reset_index().fillna(0)
more_than_mean_add_std = wa.groupby(['uid', 'lmore_than_mean_add_std'])['uid'].count().unstack().reset_index().fillna(0)

wa_down_flow_B = pd.DataFrame()
wa_down_flow_B['uid'] = wa.uid
wa_down_flow_B['wa_down_flow_less_than_mean_minus_std'] = less_than_mean_minus_std[1]
wa_down_flow_B['wa_down_flow_less_than_mean_add_std'] = less_than_mean_add_std[1]
wa_down_flow_B['wa_down_flow_more_than_mean_add_std'] = more_than_mean_add_std[1]

In [46]:
wa_day_visit_app = wa.groupby(['uid','date'])['uid'].count().unstack().add_prefix('wa_day_visit_app_').reset_index().fillna(0)

In [57]:
day = wa_day_visit_app.drop('uid', axis=1).values

In [67]:
max_apps = []
min_apps = []
mean_apps = []
median_apps = []
std_apps = []
sum_apps = []
for i in day:
    max_apps.append(max(i))
    min_apps.append(min(i))
    mean_apps.append(np.mean(i))
    median_apps.append(np.median(i))
    std_apps.append(np.std(i))
    sum_apps.append(sum(i))
#sum_apps

In [68]:
wa_day_visit_app['wa_day_visit_app_max'] = max_apps
wa_day_visit_app['wa_day_visit_app_min'] = min_apps
wa_day_visit_app['wa_day_visit_app_mean'] = mean_apps
wa_day_visit_app['wa_day_visit_app_median'] = median_apps
wa_day_visit_app['wa_day_visit_app_std'] = std_apps
wa_day_visit_app['wa_day_visit_app_std'].fillna(0)
wa_day_visit_app['wa_day_visit_app_sum'] = sum_apps
wa_day_visit_app.head()

date,uid,wa_day_visit_app_01,wa_day_visit_app_02,wa_day_visit_app_03,wa_day_visit_app_04,wa_day_visit_app_05,wa_day_visit_app_06,wa_day_visit_app_07,wa_day_visit_app_08,wa_day_visit_app_09,...,wa_day_visit_app_42,wa_day_visit_app_43,wa_day_visit_app_44,wa_day_visit_app_45,wa_day_visit_app_max,wa_day_visit_app_min,wa_day_visit_app_mean,wa_day_visit_app_median,wa_day_visit_app_std,wa_day_visit_app_sum
0,u0001,47.0,0.0,12.0,48.0,4.0,24.0,7.0,2.0,37.0,...,44.0,34.0,39.0,38.0,66.0,0.0,25.666667,29.0,18.879736,1155.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.644444,0.0,1.416656,29.0
2,u0003,20.0,20.0,27.0,20.0,33.0,29.0,49.0,31.0,23.0,...,49.0,20.0,34.0,57.0,57.0,4.0,25.933333,25.0,11.980354,1167.0
3,u0004,36.0,83.0,95.0,62.0,49.0,32.0,46.0,39.0,27.0,...,41.0,29.0,42.0,32.0,95.0,3.0,37.333333,38.0,16.907592,1680.0
4,u0005,4.0,4.0,0.0,17.0,40.0,5.0,4.0,0.0,0.0,...,12.0,21.0,33.0,16.0,58.0,0.0,14.244444,12.0,13.341923,641.0


In [75]:
feature = [voice_talk_time,voice_day_talk,voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,
           voice_in_out,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,sms_day_send,wa_name,visit_cnt,
           visit_dura,up_flow,down_flow,wa_up_flow_B,wa_downflow_B,wa_day_visit_app]

In [76]:
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

In [77]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

In [78]:
train_feature.to_csv('../data/train_featureV1.csv',index=None)
test_feature.to_csv('../data/test_featureV2.csv',index=None)